# Getting Started with BlazingSQL

[BlazingSQL](https://github.com/BlazingDB/blazingsql) provides an open-source SQL interface to ETL massive datasets directly into GPU memory and the [RAPIDS.ai](https://github.com/rapidsai) Ecosystem. 

In this notebook, we will cover how to query cuDF (GPU) DataFrames with BlazingSQL. 

To learn more about the GPU DataFrame and how it enables end-to-end workloads on RAPIDS, check out our [blog post](https://blog.blazingdb.com/blazingsql-part-1-the-gpu-dataframe-gdf-and-cudf-in-rapids-ai-96ec15102240).

## Imports

In [1]:
import cudf
from blazingsql import BlazingContext

## Connect to BlazingSQL - Create BlazingContext
You can think of the BlazingContext much like a SparkContext; this is where information such as FileSystems you have registered and Tables you have created will be stored.

In [2]:
bc = BlazingContext()

BlazingContext ready


## cuDF -> BSQL
In the next few cells, we'll genereate a cuDF DataFrame and create a BlazingSQL table from it. 

In [3]:
# generate cuDF DataFrame
df = cudf.DataFrame()

# add id & value columns
df['id'] = [1, 7, 4, 2, 9]
df['rank'] = [1, 3, 4, 3, 5]
df['score'] = ['a', 'b', 'c', 'd', 'e']

# how's it look?
df

,id,rank,score
0,1,1,a
1,7,3,b
2,4,4,c
3,2,3,d
4,9,5,e


#### Create a Table
Now we can easily create a table with BlazingContext's `.create_table()` method. 

In [4]:
# BlazingSQL table from DataFrame
bc.create_table('table_a', df)

## Query a Table
We can can now execute SQL queries with `.sql()`, which processes data on GPU and returns results as cuDF DataFrames!

In [5]:
# query everything from the first 2 instances 
bc.sql('select * from table_a LIMIT 2')

,id,rank,score
0,1,1,a
1,7,3,b


In [6]:
# query table - how many instances are there?
bc.sql('select count(*) from table_a')

,count(*)
0,5


In [7]:
# query events with a value of at least 7
bc.sql('SELECT * FROM table_a WHERE id >= 7')

,id,rank,score
0,7,3,b
1,9,5,e


# You're Ready to Rock
And... thats it! You are now live with BlazingSQL.


Check out our [docs](https://docs.blazingdb.com) to get fancy or to learn more about how BlazingSQL works with the rest of [RAPIDS AI](https://rapids.ai/).